In [ ]:
# Step 1: Install python-chess for PGN parsing
!pip install python-chess
!pip install chess

In [ ]:
import pandas as pd
import chess.pgn
from io import StringIO

# Step 2: Load the PGN file
pgn_file_path = '/content/part_2.pgn'  # Path to the uploaded PGN file
with open(pgn_file_path, 'r') as file:
    pgn_data = file.read()

# Step 3: Create tables as pandas DataFrames
players = []
games = []
moves = []

# Parse PGN data
pgn_stream = StringIO(pgn_data)
game_id_counter = 1
player_id_map = {}

while True:
    game = chess.pgn.read_game(pgn_stream)
    if game is None:
        break

    # Extract player names
    white_player = game.headers.get("White", "Unknown")
    black_player = game.headers.get("Black", "Unknown")

    # Assign player IDs
    if white_player not in player_id_map:
        player_id_map[white_player] = len(player_id_map) + 1
        players.append({"player_id": player_id_map[white_player], "name": white_player})
    if black_player not in player_id_map:
        player_id_map[black_player] = len(player_id_map) + 1
        players.append({"player_id": player_id_map[black_player], "name": black_player})

    white_id = player_id_map[white_player]
    black_id = player_id_map[black_player]

    # Extract game details
    winner_name = game.headers.get("Result", "Unknown")
    winner_id = None
    if winner_name == "1-0":
        winner_id = white_id
    elif winner_name == "0-1":
        winner_id = black_id

    # Helper function to safely convert Elo ratings
    def safe_int(value, default=0):
        try:
            return int(value)
        except ValueError:
            return default

    date = game.headers.get("Date", "????.??.??")
    if date == "????.??.??":
        date = "0000-00-00"  # Placeholder for unknown dates

    round_number = game.headers.get("Round", "?")
    if round_number == "?":
        round_number = "Unknown"

# Replace the problematic lines with a safe conversion
    games.append({
        "game_id": int(game_id_counter),
        "lichess_game_id": game.headers.get("Site", ""),
        "site": game.headers.get("Event", ""),
        "date": date,
        "round": round_number,
        "white": white_id,
        "black": black_id,
        "white_elo": safe_int(game.headers.get("WhiteElo", 0)),  # Use safe_int here
        "black_elo": safe_int(game.headers.get("BlackElo", 0)),  # Use safe_int here
        "winner": winner_id,
        "pgn": str(game),
        "eco": game.headers.get("ECO", "")
    })
    # Extract moves
    move_number = 1
    board = game.board()
    for move in game.mainline_moves():
        moves.append({
            "move_id": len(moves) + 1,
            "game_id": game_id_counter,
            "move_number": move_number,
            "move": board.san(move)
        })
        board.push(move)
        move_number += 1

    game_id_counter += 1

# Step 4: Convert tables to DataFrames
players_df = pd.DataFrame(players)
games_df = pd.DataFrame(games)
moves_df = pd.DataFrame(moves)

games_df['game_id'] = games_df['game_id'].astype(int)
moves_df['game_id'] = moves_df['game_id'].astype(int)

# Step 5: Export tables as CSV files
players_df.to_csv("Players.csv", index=False)
games_df.to_csv("Games.csv", index=False)
moves_df.to_csv("Moves.csv", index=False)

print("Data has been processed and exported as CSV files.")


Data has been processed and exported as CSV files.


In [ ]:
# Ensure the dataset has a 'rating' column for player ratings. If it doesn't exist, simulate the process.
if 'rating' in data.columns:
    # Select the relevant data for White and Black moves
    white_ratings = data[data['move_number'] == 1].groupby('move')['rating'].mean().reset_index()
    white_ratings.columns = ['Move', 'Average_Rating']
    white_ratings['Role'] = 'White'

    black_ratings = data[data['move_number'] == 2].groupby('move')['rating'].mean().reset_index()
    black_ratings.columns = ['Move', 'Average_Rating']
    black_ratings['Role'] = 'Black'

    # Combine the ratings into one DataFrame
    combined_ratings = pd.concat([white_ratings, black_ratings], axis=0)

    # Save the result for Tableau
    ratings_file_path = '/mnt/data/PlayerRatingsForOpenings.csv'
    combined_ratings.to_csv(ratings_file_path, index=False)
else:
    ratings_file_path = None

# Provide the output file path or a note if the column doesn't exist
ratings_file_path
